8.1


In [1]:
import numpy as np, random, math

x = np.array([56,101,78,67,93,87,64,72,80,69])
n, a, b, B = 10, -5, 5, 10_000
random.seed(42); np.random.seed(42)
mu_hat = x.mean()
p_hat = np.mean([
    a < np.random.choice(x, n, True).mean() - mu_hat < b
    for _ in range(B)
])
se_hat = math.sqrt(p_hat*(1-p_hat)/B)
print(p_hat, se_hat)

0.765 0.0042399882075307704


8.2

In [2]:
import numpy as np

# observed data
x = np.array([5, 4, 9, 6, 21, 17, 11, 20, 7, 10, 21, 15, 13, 16, 8])
n  = len(x)
B  = 10_000                # number of bootstrap draws
rng = np.random.default_rng(42)  # for reproducibility

# original sample variance (ddof=1 gives the usual unbiased estimator)
s2_hat = np.var(x, ddof=1)

# bootstrap loop
boot_s2 = np.empty(B)
for b in range(B):
    sample = rng.choice(x, n, replace=True)
    boot_s2[b] = np.var(sample, ddof=1)

# bootstrap estimate of Var(S^2)
var_s2_hat = np.var(boot_s2, ddof=1)

print(f"S^2 (original sample)        : {s2_hat:.4f}")
print(f"Bootstrap Var(S^2) estimate  : {var_s2_hat:.4f}")


S^2 (original sample)        : 34.3143
Bootstrap Var(S^2) estimate  : 58.4442


8.3

In [3]:
import numpy as np

def median_with_bootstrap(data, r=100, rng=None):
    """
    Return the sample median and its bootstrap variance estimate.

    Parameters
    ----------
    data : array-like
        Observed values.
    r : int, default 100
        Number of bootstrap resamples.
    rng : numpy.random.Generator, optional
        Random‐number generator for reproducibility.

    Returns
    -------
    med : float
        Sample median.
    var_boot : float
        Bootstrap estimate of Var(median).
    """
    rng = rng or np.random.default_rng()
    data = np.asarray(data)
    n = len(data)

    med = np.median(data)

    # generate r bootstrap samples of size n
    boot_samples = rng.choice(data, size=(r, n), replace=True)
    boot_medians = np.median(boot_samples, axis=1)

    var_boot = np.var(boot_medians, ddof=1)
    return med, var_boot


def var_bootstrap_mean(data, r=100, rng=None):
    """Bootstrap estimate of Var(sample mean)."""
    rng = rng or np.random.default_rng()
    data = np.asarray(data)
    n = len(data)

    boot_means = rng.choice(data, size=(r, n), replace=True).mean(axis=1)
    return np.var(boot_means, ddof=1)


# ----- simulate the data ----------------------------------------------------
rng   = np.random.default_rng(42)  # fixed seed for repeatability
N     = 200
beta  = 1.0          # scale parameter
k     = 1.05         # shape parameter
data  = beta * (rng.pareto(k, N) + 1.0)   # Pareto(β=1, k=1.05)

# ----- (a) sample mean and median -------------------------------------------
sample_mean   = data.mean()
sample_median, var_median_boot = median_with_bootstrap(data, r=100, rng=rng)

# ----- (b) bootstrap Var(mean) ----------------------------------------------
var_mean_boot = var_bootstrap_mean(data, r=100, rng=rng)

# ----- (c) bootstrap Var(median) is already computed above -------------------

# ----- (d) print everything --------------------------------------------------
print(f"(a) mean   = {sample_mean:.4f}")
print(f"(a) median = {sample_median:.4f}\n")

print(f"(b) bootstrap Var(mean)   = {var_mean_boot:.4f}")
print(f"(c) bootstrap Var(median) = {var_median_boot:.4f}\n")

prec_mean   = 1 / var_mean_boot
prec_median = 1 / var_median_boot
print(f"(d) precision: 1/Var(mean)   = {prec_mean:.4f}")
print(f"    precision: 1/Var(median) = {prec_median:.4f}")


(a) mean   = 5.1809
(a) median = 1.9199

(b) bootstrap Var(mean)   = 1.5963
(c) bootstrap Var(median) = 0.0363

(d) precision: 1/Var(mean)   = 0.6265
    precision: 1/Var(median) = 27.5846
